# Comparison of VCF files

## 1. Introduction

The goal of this assignment is to do variant calling on human_g1k_v37_decoy.fasta reference genome and C835.HCC1143.2.converted.realigned.base_recalibrated.bam file using two callers: GATK 4 HaplotypeCaller [1] and FreeBayes [2].

After that, results need to be compared using HaplotypeCaller's one as a truth set and Freebayes' one as a test set. Informations of interest are: number of true positives, false positives and false negatives, precision, recall and f-score.

## 2. Variant calling

In order to do variant calling with GATK 4 HaplotypeCaller, two input arguments are required: input alingments (BAM file) and reference (FASTA file). Apart from those, two additional files need to be present in the directory with reference files, with FAI and DICT formats. Results are written into gatk_result.vcf, and this will be considered the truth set of the further analysis.

FreeBayes also requires same arguments as previous caller, but it does not require any additional files to be present. Results are written into freebayes_result.freebayes.vcf, which will be test set of the further analysis.

## References

[1] https://gatk.broadinstitute.org/hc/en-us

[2] https://github.com/freebayes/freebayes